In [1]:
import evalml
from evalml import AutoMLSearch
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
# data = evalml.demos.load_fraud(n_rows=1000)
data= pd.read_csv('Telco-Customer-Churn.csv')
data.head()

In [2]:
## Split the data into training and testing sets
X, y = data.drop(columns=['Churn']), data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Specify the objective to optimize (e.g., 'F1 Macro' for classification)
# objective = 'F1 Macro'   # used for multi_classification
objective = 'f1'

# Create and run the AutoML search
# automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary', objective=objective)
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary', objective=objective, max_iterations=20)
automl.search()

{1: {'Random Forest Classifier w/ Label Encoder + Drop Columns Transformer + Imputer + One Hot Encoder + RF Classifier Select From Model': 3.683878183364868,
  'Total time of batch': 4.501164674758911},
 2: {'LightGBM Classifier w/ Label Encoder + Select Columns By Type Transformer + Label Encoder + Drop Columns Transformer + Imputer + Select Columns Transformer + Select Columns Transformer + Label Encoder + Imputer + One Hot Encoder': 3.546255588531494,
  'Extra Trees Classifier w/ Label Encoder + Select Columns By Type Transformer + Label Encoder + Drop Columns Transformer + Imputer + Select Columns Transformer + Select Columns Transformer + Label Encoder + Imputer + One Hot Encoder': 3.319422721862793,
  'Elastic Net Classifier w/ Label Encoder + Select Columns By Type Transformer + Label Encoder + Drop Columns Transformer + Imputer + Standard Scaler + Select Columns Transformer + Select Columns Transformer + Label Encoder + Imputer + One Hot Encoder + Standard Scaler': 3.2041699886

In [5]:

# Access the ranking of pipelines
pipeline_rankings = automl.rankings
pipeline_rankings

,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,6,Logistic Regression Classifier w/ Label Encode...,6,0.623681,0.623681,0.012150,62.368143,False,"{'Label Encoder': {'positive_label': None}, 'N..."
2,11,Random Forest Classifier w/ Label Encoder + Se...,11,0.622581,0.622581,0.009730,62.258083,False,"{'Label Encoder': {'positive_label': None}, 'N..."
8,1,Random Forest Classifier w/ Label Encoder + Dr...,1,0.619264,0.619264,0.012222,61.926433,False,"{'Label Encoder': {'positive_label': None}, 'D..."
9,4,Elastic Net Classifier w/ Label Encoder + Sele...,4,0.618469,0.618469,0.010947,61.846927,False,"{'Label Encoder': {'positive_label': None}, 'N..."
10,9,Extra Trees Classifier w/ Label Encoder + Sele...,9,0.618129,0.618129,0.007668,61.812923,False,"{'Label Encoder': {'positive_label': None}, 'N..."
16,5,XGBoost Classifier w/ Label Encoder + Select C...,5,0.605595,0.605595,0.004872,60.559487,False,"{'Label Encoder': {'positive_label': None}, 'N..."
18,2,LightGBM Classifier w/ Label Encoder + Select ...,2,0.598948,0.598948,0.006614,59.894789,False,"{'Label Encoder': {'positive_label': None}, 'N..."
19,0,Mode Baseline Binary Classification Pipeline,0,0.000000,0.000000,0.000000,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [6]:

# Retrieve the best pipeline
best_pipeline = automl.best_pipeline
best_pipeline


pipeline = BinaryClassificationPipeline(component_graph={'Label Encoder': ['Label Encoder', 'X', 'y'], 'Numeric Pipeline - Select Columns By Type Transformer': ['Select Columns By Type Transformer', 'X', 'Label Encoder.y'], 'Numeric Pipeline - Label Encoder': ['Label Encoder', 'Numeric Pipeline - Select Columns By Type Transformer.x', 'Label Encoder.y'], 'Numeric Pipeline - Drop Columns Transformer': ['Drop Columns Transformer', 'Numeric Pipeline - Select Columns By Type Transformer.x', 'Numeric Pipeline - Label Encoder.y'], 'Numeric Pipeline - Imputer': ['Imputer', 'Numeric Pipeline - Drop Columns Transformer.x', 'Numeric Pipeline - Label Encoder.y'], 'Numeric Pipeline - Standard Scaler': ['Standard Scaler', 'Numeric Pipeline - Imputer.x', 'Numeric Pipeline - Label Encoder.y'], 'Numeric Pipeline - Select Columns Transformer': ['Select Columns Transformer', 'Numeric Pipeline - Standard Scaler.x', 'Numeric Pipeline - Label Encoder.y'], 'Categorical Pipeline - Select Columns Transformer'

In [7]:
# Retrieve the best pipeline and summarize the results
best_pipeline = automl.best_pipeline
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


***************************************************************************************************************************************************************************************************************************************************************************************
* Logistic Regression Classifier w/ Label Encoder + Select Columns By Type Transformer + Label Encoder + Drop Columns Transformer + Imputer + Standard Scaler + Select Columns Transformer + Select Columns Transformer + Label Encoder + Imputer + One Hot Encoder + Standard Scaler *
***************************************************************************************************************************************************************************************************************************************************************************************

Problem Type: binary
Model Family: Linear

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Select Columns By Type Transformer
	 * column_types : 

# Save the best pipeline


In [8]:
# Save the best pipeline
best_pipeline.save("Binary_Cassification_model.pkl")

# Loading the saved model in order to do prediction

In [9]:
final_model=automl.load('Binary_Cassification_model.pkl')

# Make predictions on the test set


In [10]:
# Make predictions on the test set
y_pred = final_model.predict(X_test)
y_pred

185      True
2715    False
3825    False
1807     True
132     False
        ...  
6366    False
315     False
2439    False
5002    False
1161     True
Name: Churn, Length: 1409, dtype: bool

In [11]:
df_pred=pd.DataFrame({'True_result': y_test, 'Predicted_result':y_pred})
# res={'True':'Yes','False':'No'}
# res={'Yes':'True','No':'False'}
df_pred['Predicted_result']=np.where(df_pred['Predicted_result']==True,'Yes','No')
df_pred['Correct_result']=np.where(df_pred['True_result']==df_pred['Predicted_result'],1,0)

df_pred

,True_result,Predicted_result,Correct_result
185,Yes,Yes,1
2715,No,No,1
3825,No,No,1
1807,Yes,Yes,1
132,No,No,1
...,...,...,...
6366,No,No,1
315,No,No,1
2439,No,No,1
5002,No,No,1


In [12]:
df_pred['Correct_result'].value_counts()

1    1109
0     300
Name: Correct_result, dtype: int64

In [21]:
# Evaluate the predictions
print(f"Classification Report:\n{best_pipeline.score(X_test, y_test, objectives=[objective])}")

Classification Report:
OrderedDict([('F1', 0.6470588235294117)])


In [23]:
# Access the ranking and evaluation results of pipelines
pipeline_rankings = automl.rankings
pipeline_results = automl.results

# Display the ranking along with evaluation results
for _, row in pipeline_rankings.iterrows():
    # print(row)
    pipeline_name = row['pipeline_name']
    
    # # Access and print evaluation results for each pipeline
    result_row = pipeline_results[pipeline_results['pipeline_name'] == pipeline_name].iloc[0]
    # print(result_row['id'])
    # print(result_row['score'])
    # print(result_row['percent_better_than_baseline'])
    # print(result_row['training_time'])

KeyError: 'pipeline_name'